# AlphaQUBO scipy sparse matrix

This tutorial shows how to read a scipy sparse Q matrix, compress and upload to S3, submit to an AlphaQUBO solver and retrieve the results from S3.

In [1]:
import sys
import alphaqubo_client as aq
from alphaqubo_client.rest import ApiException
import boto3
from botocore.exceptions import ClientError
import gzip
import json

## Upload File to S3

In [2]:
def upload_file(bucket, file_name, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name, ExtraArgs={'ContentType': 'application/x-gzip'})
    except ClientError as e:
        print(e)
        return False
    return True
    
def download_file( bucket, file_name):
    s3 = boto3.client('s3')
    s3.download_file(bucket, file_name, file_name)

## Save AlphaQUBO Formatted File as gzip compressed

In [3]:
def save_alphaqubo_gzip(filename:str, coomat):
    with gzip.open( filename, "wt" ) as fl:        
        count = int(0)
        nVariables = int(0)
        for i,j,d in zip(coomat.row, coomat.col, coomat.data):
            count = count + 1
            iVar = i + 1
            jVar = j + 1
            if iVar > nVariables:
                nVariables = iVar
            if jVar > nVariables:
                nVariables = jVar

        print('p', nVariables, count, file=fl)

        for i,j,d in zip(coomat.row, coomat.col, coomat.data):
            iVar = i + 1
            jVar = j + 1
            if i != j:
                d = d / 2
            print(iVar, jVar, d, file=fl)

## Submit Async AlphaQUBO job

In [4]:
def submit_async(region, bucket_name, filename, maximize=False, time_limit=30 ):            
        body = aq.SolverAsyncRequest()
        body.num_vars = 0
        body.region = region
        body.bucket_name = bucket_name
        body.key_name = filename
        body.solution_bucket_name = bucket_name
        body.solution_key_name = filename + '.sol'
        
        if maximize:
            body.min_max = 1
        else:
            body.min_max = 0
        body.timeout = time_limit
        
        try:
            # solve it asynchronously.
            api_response = api_instance.api_qubo_solve_qubo_async_using_s3_post(body=body)           
        except ApiException as e:
            print("Exception when calling QuboApi->api_qubo_solve_qubo_async_using_s3_post: %s\n" % e)
        
        return api_response

## Get the results of previously submitted job

In [5]:
def job_results(bucket_name, filename):
    solfile = filename + '.sol'
    download_file(bucket_name, solfile)
    with open(solfile) as f:
        data = json.load(f)
    return data

## Configure AlphaQUBO API
Configure the connection details to connect to AlphaQUBO SolverAPI

In [6]:
configuration = aq.Configuration()
configuration.debug = False
configuration.host = "http://ec2-34-223-111-151.us-west-2.compute.amazonaws.com:9000"

api_instance = aq.QuboApi(aq.ApiClient(configuration))

### Read Saved NPZ file from scipy.sparse


In [7]:
BUCKET_NAME = 'alphaqubo-test'
FILENAME = 'test.gz'

from scipy.sparse import coo_matrix, save_npz, load_npz

mat = load_npz('../data/test.npz')
print("Number of Variables: ", mat.shape[0])
print("Saving compressed qubo")
#save_alphaqubo_gzip(FILENAME, mat)
print("Uploading File to S3")
upload_file(BUCKET_NAME, FILENAME)

Number of Variables:  21943
Saving compressed qubo
Uploading File to S3


True

## Submit Async Job to AlphaQUBO

In [8]:
submit_async('us-west-2', BUCKET_NAME, FILENAME, time_limit=300 )

{'bucket_name': 'alphaqubo-test',
 'qubo_accepted': True,
 'qubo_key_name': 'test.gz',
 'region': 'us-west-2',
 'solution_bucket_name': 'alphaqubo-test',
 'solution_httpurl': 'https://s3.us-west-2.amazonaws.com/alphaqubo-test/test.gz.sol',
 'solution_key_name': 'test.gz.sol',
 'solution_s3_uri': 's3://alphaqubo-test/test.gz.sol'}

### AlphaQUBO Retrieve Results

In [10]:
job_results(BUCKET_NAME, FILENAME)

{'numVars': 21943,
 'nonZero': 3419670,
 'timeout': 300,
 'minMax': 0,
 'parameters': None,
 'solvedValue': -454.0110000000001,
 'solvedResult': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
 